In [ ]:
# import websocket
# import json
# import threading
# import time
# from datetime import datetime, timezone, timedelta
# import matplotlib.pyplot as plt
# from IPython.display import clear_output, display

# timestamps = []
# prices = []
# running = True

# def format_sgt_time(ts):
#     """Convert UNIX timestamp to Singapore Time (HH:MM:SS)."""
#     sgt = timezone(timedelta(hours=8))
#     return datetime.fromtimestamp(int(ts), tz=timezone.utc).astimezone(sgt).strftime("%H:%M:%S")

# def on_open(ws):
#     print("Connected to Bitstamp BTC/USD WebSocket")
#     ws.send(json.dumps({
#         "event": "bts:subscribe",
#         "data": {"channel": "live_trades_btcusd"}
#     }))

# def on_message(ws, message):
#     data = json.loads(message)
#     if data.get("event") == "trade":
#         trade = data["data"]
#         price = float(trade["price"])
#         t = format_sgt_time(trade["timestamp"])
#         timestamps.append(t)
#         prices.append(price)

# def on_error(ws, error):
#     print("Error:", error)

# def on_close(ws, code, msg):
#     print("Disconnected from Bitstamp WebSocket")

# def run_ws():
#     socket = "wss://ws.bitstamp.net"
#     ws = websocket.WebSocketApp(socket,
#                                 on_open=on_open,
#                                 on_message=on_message,
#                                 on_close=on_close,
#                                 on_error=on_error)
#     ws.run_forever()

# # Start the WebSocket in a background thread
# t = threading.Thread(target=run_ws)
# t.daemon = True
# t.start()

# # Live plot loop
# plt.ion()
# plt.style.use("seaborn-v0_8-whitegrid")
# fig, ax = plt.subplots(figsize=(8, 5))

# try:
#     while True:
#         if prices:
#             clear_output(wait=True)
#             ax.clear()
#             ax.plot(timestamps, prices, linewidth=2, color="#0077cc")
#             ax.set_title("💹 Live BTC/USD Price (Bitstamp)", fontsize=14, pad=12)
#             ax.set_xlabel("Time (SGT)", fontsize=10)
#             ax.set_ylabel("Price (USD)", fontsize=10)
#             ax.tick_params(axis='x', rotation=45)
#             ax.grid(True, linestyle="--", alpha=0.4)
#             ax.set_facecolor("#fafafa")
#             plt.tight_layout()
#             display(fig)
#             time.sleep(2)
# except KeyboardInterrupt:
#     running = False
#     print("Stopped chart.")


: 

In [13]:
import asyncio
import aiohttp
from datetime import datetime

class WhaleAlertBot:
    def __init__(self, min_btc=50):
        self.min_sats = int(min_btc * 100_000_000)
        self.min_btc = min_btc
        self.session = None
        self.btc_price = None
        self.api_url = "https://mempool.space/api/mempool/recent"
        self.checked_tx = set()

    async def init_session(self):
        if not self.session:
            self.session = aiohttp.ClientSession()

    async def close_session(self):
        if self.session:
            await self.session.close()

    async def get_btc_price(self):
        try:
            await self.init_session()
            async with self.session.get(
                "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
            ) as resp:
                data = await resp.json()
                self.btc_price = data["bitcoin"]["usd"]
        except:
            self.btc_price = 30_000
        return self.btc_price
    
    async def price_updater(self):
        while True:
            await self.get_btc_price()
            await asyncio.sleep(5)  # every 5 minutes


    async def send_alert(self, txid, value):
        btc = value / 100_000_000
        usd = btc * (self.btc_price or 30_000)
        print("=" * 80)
        print(f"🐋  Whale transaction detected!")
        print(f"💎  {btc:,.6f} BTC (${usd:,.0f})")
        print(f"🔗  https://mempool.space/tx/{txid}")
        print("=" * 80 + "\n")

    async def monitor(self):
        print(f"🚀 Monitoring mempool.space for transactions ≥ {self.min_btc} BTC...")
        await self.get_btc_price()
        asyncio.create_task(self.price_updater())  # run in background

        while True:
            try:
                await self.init_session()
                async with self.session.get(self.api_url) as resp:
                    txs = await resp.json()

                for tx in txs:
                    txid = tx["txid"]
                    value = tx.get("value", 0)
                    if value >= self.min_sats and txid not in self.checked_tx:
                        self.checked_tx.add(txid)
                        await self.send_alert(txid, value)
                
                if len(self.checked_tx) > 5000:
                    self.checked_tx.clear()

            except Exception as e:
                print(f"⚠️ Error fetching transactions: {e}")

            await asyncio.sleep(10)  # Poll every 10 seconds

async def main():
    bot = WhaleAlertBot(min_btc=0.01)
    try:
        await bot.monitor()
    except KeyboardInterrupt:
        print("\n👋 Stopping bot...")
    finally:
        await bot.close_session()

if __name__ == "__main__":
    asyncio.run(main())


🚀 Monitoring mempool.space for transactions ≥ 0.01 BTC...
🐋  Whale transaction detected!
💎  0.078876 BTC ($9,045)
🔗  https://mempool.space/tx/93662f0a702bd984722596154aa214dc3c3ae518b3099d470a6e5360b0f746e2

🐋  Whale transaction detected!
💎  0.279461 BTC ($32,048)
🔗  https://mempool.space/tx/2bcd731d89f5e2e8da9ea6f9b5c6ddf63fd0637b8042aa871d445e97b2967edc

🐋  Whale transaction detected!
💎  0.364898 BTC ($41,846)
🔗  https://mempool.space/tx/1efa413ba633ea70198d0273728b9096149da25638507c967d87ae9badf9f48f

🐋  Whale transaction detected!
💎  0.423351 BTC ($48,549)
🔗  https://mempool.space/tx/0307dbd80521cbffbd9d5c8b9c33907dce77c7c03af5067d3354f838885e6692

🐋  Whale transaction detected!
💎  2.435970 BTC ($279,355)
🔗  https://mempool.space/tx/57f072e84551d89cececdac0f00b248bf8bfd2961792eedacd1789f758bbd62e

🐋  Whale transaction detected!
💎  0.244609 BTC ($28,052)
🔗  https://mempool.space/tx/fc1ee84b5c6d08cdbecc488aa8c6574dcadb195239dc6eea50cc8997975d08d4

🐋  Whale transaction detected!
💎  0.

KeyboardInterrupt: 